In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
# 지하철 인접행렬csv 불러오기
df1 = pd.read_csv('./인접행렬1.csv')
df1 = df1.drop(columns='0', axis =1)
df1

FileNotFoundError: ignored

In [ ]:
data1 = df1.to_numpy()
data1[40][131]

In [ ]:
# 최소환승을 위한 데이터 (환승시간 +30)
df2 = pd.read_csv('./인접행렬2.csv')
df2 = df2.drop(columns='0', axis =1)
df2

,소요산,동두천,보산,동두천중앙,지행,덕정,덕계,양주,녹양,가능,...,하남풍산,하남시청,하남검단산,둔촌동,올림픽공원,방이,오금.1,개롱,거여,마천
0,0,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,2,0,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,2,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,1,0,4,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
297,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
298,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,1,0
299,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1


In [ ]:
data2 = df2.to_numpy()
data2[40][131]

32

In [ ]:
# HMatrix
df3 = pd.read_csv('./HMatrix.csv')
df3 = df3.drop(columns='0', axis =1)
data3 = df3.to_numpy()

In [ ]:
# 지하철 이름 리스트 형태로 불러오기 
name = pd.read_csv('./지하철.csv', encoding = 'cp949')
name = name.transpose()
name = name.values.tolist()
name = sum(name, [])

# 입력값: 최소환승 / 최소시간 선택
print("최소환승 or 최소시간 선택: ", end = '')

def sel(s, path, time):
    s = str(s)
    if s == '최소환승':
        data = path
        
    elif s == '최소시간':
        data = time
        
    return data

data = sel(input(), data2, data1)

# 입력값: 출발역
print("출발역(호선,역이름): ", end = '')
a = input()
start = int(name.index(a))

# 입력값: 도착역
print("도착역(호선,역이름): ", end = '')
b = input()
dest = int(name.index(b))

class Graph:
    # vertex : Queue형태
    def minTime(self,time,queue):
        # minimum : 무한대로 초기화
        # minimun index는 음수인 -1로 초기화
        minimum = float("Inf")
        min_index = -1
         
        # minimum 갖는 값 가져오기 
        for i in range(len(time)):
            if time[i] < minimum and i in queue:
                minimum = time[i]
                min_index = i
        return min_index
 

    # shortest path 출력하기
    # parent = parent array, j = source
    def getPath(self, parent, j, out):
        
        #Base Case : If j is start point
        if parent[j] == -1 :
            out.append(name[j])
            return 
        
        else :
            self.getPath(parent, parent[j], out)
            out.append(name[j])
 

    def dijkstra(self, graph, start, out):
 
        row = len(graph)
        col = len(graph[0])
 
        # 모든 time 무한대로 초기화
        time = [float("Inf")] * row
 
        # shortest path 저장하는 parent array 음수로 초기화
        parent = [-1] * row
 
        # start = 출발역
        time[start] = 0
     
        # 모든 역들 queue에 넣어주기
        queue = []
        for i in range(row):
            queue.append(i)
             
        # 모든 역들에 대해 shortest path 찾기
        while queue:
 
            # Queue 속 vertice(역)에 대해 minTime 갖는 index 반환
            u = self.minTime(time,queue)
 
            # Queue에서 u 제거    
            queue.remove(u)
 
            # time value & parent index update
            for i in range(col):
                # Queue 속에 있으면서
                # u에서 i까지의 edge가 있고, 출발역에서 i를 통해 u까지의 경로 중
                # 총 시간이 time[i] 값보다 작은 경우 time[i] update
                if graph[u][i] and i in queue:
                    if time[u] + graph[u][i] < time[i]:
                        time[i] = time[u] + graph[u][i]
                        parent[i] = u
 
        self.getPath(parent, dest, out)
       
        print("경로:", end=' ')     
        # 경로에 색입히기
        for i in range(len(out)):
            x = out[i][0]
            if x == '1':
                color = 34
            elif x == '2':
                color = 32
            elif x == '3':
                color = 91
            elif x == '4':
                color = 96
            else:
                color = 35
            color_f = '\033[' + str(color) + 'm'
            print( color_f,out[i],'\033[30m','-',end=' ')     
            
        print('도착!')
        
        # 최소환승: 총 시간에서 환승 횟수*30분 빼주기
        if graph[40][131] == 32:
            count = 0
            for i in range(len(out) - 1):
                if out[i][0] != out[i+1][0]:
                    count += 1
            print('소요시간:', time[dest] - 30*count, '분')
            
        # 최소시간: 총 시간 그대로 출력    
        elif graph[40][131] == 2:
            print('소요시간:', time[dest], '분')
   
        
    def a_star(self, graph, heuristic, start, dest, out):
 
        row = len(graph)
        col = len(graph[0])
 
        # 모든 time 무한대로 초기화
        time = [float("Inf")] * row
 
        # start = 출발역
        time[start] = 0
        
        # shortest path 저장하는 parent array 음수로 초기화
        parent = [-1] * row
        
        priorities = [float("Inf")] * row
        priorities[start] = heuristic[start][dest] 
        visited = [False] * row
             
        # 모든 역들에 대해 shortest path 찾기
        while True:
            lowest_priority = float("inf")
            lowest_priority_index = -1
            for i in range(len(priorities)):
                # 최초 방문 노드 
                if priorities[i] < lowest_priority and not visited[i]:
                    lowest_priority = priorities[i]
                    lowest_priority_index = i

            if lowest_priority_index == -1:
                # Node not found
                return -1

            elif lowest_priority_index == dest:
                
                self.getPath(parent, dest, out)
       
                print("경로:", end=' ')     
                # 경로에 색입히기
                for i in range(len(out)):
                    x = out[i][0]
                    if x == '1':
                        color = 34
                    elif x == '2':
                        color = 32
                    elif x == '3':
                        color = 91
                    elif x == '4':
                        color = 96
                    else:
                        color = 35
                        
                    color_f = '\033[' + str(color) + 'm'
                    print( color_f,out[i],'\033[30m','-',end=' ')     
            
                print('도착!')
                
                # 최소환승: 총 시간에서 환승 횟수*30분 빼주기
                if graph[40][131] == 32:
                    count = 0
                    for i in range(len(out) - 1):
                        if out[i][0] != out[i+1][0]:
                            count += 1
                    print('소요시간:', time[dest] - 30*count, '분')

                # 최소시간: 총 시간 그대로 출력    
                elif graph[40][131] == 2:
                    print('소요시간:', time[dest], '분')
            
                # Goal node found
                return -1 

            visited[lowest_priority_index] = True
 
            # time value & parent index update
            for i in range(len(graph[lowest_priority_index])):
                if graph[lowest_priority_index][i] != 0 and not visited[i]:
                    if time[lowest_priority_index] + graph[lowest_priority_index][i] < time[i]:
                        time[i] = time[lowest_priority_index] + graph[lowest_priority_index][i]
                        priorities[i] = time[i] + heuristic[i][dest]
                        parent[i] = lowest_priority_index
                        
        
                        
    
g = Graph() 
graph = data
out = []
#A* 알고리즘을 위한 HMatrix 
h = data3
    
# 성능 비교를 위한 시간 측정    
Stime = time.time()

########## 함수 실행 (Dijkstra or A*) ##########
g.dijkstra(graph, start, out)
#g.a_star(graph, h, start, dest, out)

Ftime = time.time()
print("성능(s):", Ftime - Stime)

최소환승 or 최소시간 선택: 최소환승
출발역(호선,역이름): 1서동탄
도착역(호선,역이름): 5길동
경로:  1서동탄  -  1병점  -  1세류  -  1수원  -  1화서  -  1성균관대  -  1의왕  -  1당정  -  1군포  -  1금정  -  1명학  -  1안양  -  1관악  -  1석수  -  1금천구청  -  1독산  -  1가산디지털단지  -  1구로  -  1신도림  -  1영등포  -  1신길  -  5신길  -  5여의도  -  5여의나루  -  5마포  -  5공덕  -  5애오개  -  5충정로  -  5서대문  -  5광화문  -  5종로3가  -  5을지로4가  -  5동대문역사문화공원  -  5청구  -  5신금호  -  5행당  -  5왕십리  -  5마장  -  5답십리  -  5장한평  -  5군자  -  5아차산  -  5광나루  -  5천호  -  5강동  -  5길동  - 도착!
소요시간: 93 분
성능(s): 0.22099947929382324
